# Customer Service Chatbot with DialogGPT for Conversational Intents

This Jupyter Notebook updates `evaluate_chatbot_data_dialoggpt_twitter_trained.ipynb` to use the latest module versions (as of May 29, 2025) for a trucking company chatbot, fixing syntax errors. It uses Microsoft’s DialogGPT for conversational intents (greeting, farewell, small_talk, compliment, weather_query) and DistilBERT for trucking-specific intents (delivery_status, billing_issue, account_update, service_inquiry, fuel_card_query, general_query). Trains on Twitter dataset (`tweets.csv` or `trucking_chatbot_test_dataset.csv`) with interactive ipywidgets UI.

## Objectives
- Inspect Twitter dataset for intents and entities (e.g., location, company).
- Train DistilBERT for intent classification.
- Fine-tune DialogGPT on conversational Twitter data.
- Evaluate DistilBERT with accuracy, F1-score, confusion matrix, and dialogue success rate.
- Implement hybrid dialogue management with DialogGPT and DistilBERT.
- Provide interactive UI for customer interaction.

## Requirements
- Python 3.8 (recommended; 3.9 also compatible)
- Install: `pip install transformers==4.44.2 torch==2.5.0 pandas==2.2.3 numpy==2.1.1 scikit-learn==1.5.2 datasets==3.0.1 seaborn==0.13.2 matplotlib==3.9.2 ipywidgets==8.1.5`
- For GPU: `pip install torch==2.5.0+cu121 --index-url https://download.pytorch.org/whl/cu121`
- Place `trucking_chatbot_test_dataset.csv` or `tweets.csv` in the directory.
- Enable widgets: `jupyter nbextension enable --py widgetsnbextension`

## Notes
- Dataset: https://www.kaggle.com/thoughtvector/customer-support-on-twitter
- Reflects tariffs/Moody’s downgrade in billing/fuel inquiries.
- Professional responses with dynamic DialogGPT conversation.
- Date: May 29, 2025, 11:56 AM EDT


## Step 1: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset
import torch
import json
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import defaultdict
import random
import re
from datetime import datetime
import os
%matplotlib inline


## Step 2: Inspect and Preprocess Dataset

Load Twitter dataset, add conversational examples, label intents/entities.

In [ ]:
def inspect_dataset(file_path='trucking_chatbot_test_dataset.csv'):
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print('Dataset not found. Using Twitter dataset.')
        try:
            df = pd.read_csv('tweets.csv')
        except FileNotFoundError:
            print('Download from: https://www.kaggle.com/thoughtvector/customer-support-on-twitter')
            data = {
                'text': [
                    'Where is my shipment from Speedway?',
                    'Why is my Comdata bill so high?',
                    'Need to update my address for IFTA',
                    'What are your hauling rates?',
                    'Help with my account',
                    'Hello',
                    'How are you',
                    'How can I help you',
                    'Track my cargo',
                    'Overcharged on Comdata invoice',
                    'Hi there',
                    'Change my contact info',
                    'Tell me about Comdata services',
                    'Lost my shipment',
                    'Good morning',
                    'Thanks for your help',
                    'Bye',
                    'What’s new?',
                    'How’s it going?',
                    'How’s your day going?',
                    'Any big plans?',
                    'How’s the trucking life?',
                    'You’re awesome!',
                    'Great job!',
                    'How’s the weather there?',
                    'Is it raining?'
                ],
                'intent': [
                    'delivery_status',
                    'billing_issue',
                    'account_update',
                    'service_inquiry',
                    'general_query',
                    'greeting',
                    'greeting',
                    'greeting',
                    'delivery_status',
                    'billing_issue',
                    'greeting',
                    'account_update',
                    'fuel_card_query',
                    'general_query',
                    'greeting',
                    'farewell',
                    'farewell',
                    'small_talk',
                    'small_talk',
                    'small_talk',
                    'small_talk',
                    'small_talk',
                    'compliment',
                    'compliment',
                    'weather_query',
                    'weather_query'
                ]
            }
            df = pd.DataFrame(data)

    keywords = ['delivery', 'shipment', 'cargo', 'bill', 'invoice', 'payment', 'account', 'service', 'hauling', 'truck', 'comdata', 'ifta', 'speedway', 'fuel', 'tax', 'station', 'hello', 'hi', 'how', 'good', 'bye', 'thanks', 'new', 'day', 'plans', 'awesome', 'great', 'weather', 'rain']
    if 'text' in df.columns:
        df = df[df['text'].str.contains('|'.join(keywords), case=False, na=False)]
        df = df.sample(n=min(10000, len(df)), random_state=42)

    if 'intent' not in df.columns:
        def label_intent(text):
            text = text.lower()
            if any(word in text for word in ['hello', 'hi', 'how are you', 'how can i help', 'good morning', 'good afternoon']):
                return 'greeting'
            elif any(word in text for word in ['goodbye', 'bye', 'thanks', 'thank you']):
                return 'farewell'
            elif any(word in text for word in ['what’s new', 'how’s it going', 'how’s your day', 'any big plans', 'trucking life']):
                return 'small_talk'
            elif any(word in text for word in ['awesome', 'great job', 'you rock']):
                return 'compliment'
            elif any(word in text for word in ['weather', 'rain', 'sunny']):
                return 'weather_query'
            elif any(word in text for word in ['delivery', 'shipment', 'track', 'cargo']):
                return 'delivery_status'
            elif any(word in text for word in ['bill', 'invoice', 'payment', 'charge']):
                return 'billing_issue'
            elif any(word in text for word in ['update', 'change', 'address', 'contact']):
                return 'account_update'
            elif any(word in text for word in ['service', 'rate', 'hauling']):
                return 'service_inquiry'
            elif any(word in text for word in ['comdata', 'ifta', 'fuel', 'tax']):
                return 'fuel_card_query'
            else:
                return 'general_query'
        df['intent'] = df['text'].apply(label_intent)

    def extract_entities(text):
        entities = []
        text = text.lower()
        if 'speedway' in text:
            entities.append({'entity': 'location', 'value': 'Speedway'})
        if 'comdata' in text:
            entities.append({'entity': 'company', 'value': 'Comdata'})
        if 'ifta' in text:
            entities.append({'entity': 'regulation', 'value': 'IFTA'})
        return entities

    df['entities'] = df['text'].apply(extract_entities)

    print('Dataset Shape:', df.shape)
    print('Columns:', df.columns.tolist())
    print('Sample Rows:\n', df.head())
    print('Missing Values:\n', df.isnull().sum())
    print('Intent Distribution:\n', df['intent'].value_counts())
    print('Entity Samples:\n', df[df['entities'].apply(len) > 0][['text', 'entities']].head())
    return df

df = inspect_dataset()


## Step 3: Prepare DialogGPT Training Data


In [ ]:
def prepare_dialoggpt_training_data(df):
    conversational_intents = ['greeting', 'farewell', 'small_talk', 'compliment', 'weather_query']
    df_conversational = df[df['intent'].isin(conversational_intents)][['text', 'intent']]

    response_map = {
        'greeting': [
            'Hello! How can I assist you with your trucking needs today?',
            'Hi there! Ready to help with your shipments or account!',
            'Good to hear from you! What’s up?'
        ],
        'farewell': [
            'Goodbye! Stay safe on the road.',
            'Thanks for connecting! Catch you later.'
        ],
        'small_talk': [
            'My day’s going smoothly, thanks! How’s yours?',
            'Trucking life’s always moving! How’s it treating you?',
            'Just keeping the wheels turning! Got any big plans?',
            'All’s good here! What’s new with you?'
        ],
        'compliment': [
            'Thanks, you’re awesome too! Need help with anything?',
            'Appreciate that! What can I do for you today?'
        ],
        'weather_query': [
            'Can’t see the skies, but I can check your route! Where are you headed?',
            'Weather’s a mystery here, but I’m ready to help! What’s your destination?'
        ]
    }

    dialogues = []
    for _, row in df_conversational.iterrows():
        user_input = row['text']
        intent = row['intent']
        response = random.choice(response_map[intent])
        dialogues.append({'input': user_input, 'response': response})

    dialogue_df = pd.DataFrame(dialogues)
    dataset = Dataset.from_pandas(dialogue_df)
    os.makedirs('data', exist_ok=True)
    dialogue_df.to_csv('data/dialoggpt_dialogues.csv', index=False)

    print(f'DialogGPT training data prepared: {len(dialogue_df)} dialogue pairs.')
    return dataset

dialoggpt_dataset = prepare_dialoggpt_training_data(df)


## Step 4: Fine-Tune DialogGPT


In [ ]:
def fine_tune_dialoggpt(dataset):
    model_name = 'microsoft/DialoGPT-medium'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token  # Fix padding token
    model = AutoModelForCausalLM.from_pretrained(model_name)

    def preprocess_dialogues(examples):
        conversations = [f"{inp} {tokenizer.eos_token} {resp}" for inp, resp in zip(examples['input'], examples['response'])]
        tokenized = tokenizer(
            conversations,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        tokenized['labels'] = tokenized['input_ids'].clone()
        return tokenized

    tokenized_dataset = dataset.map(preprocess_dialogues, batched=True)
    tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

    training_args = TrainingArguments(
        output_dir='./dialoggpt_results',
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir='./dialoggpt_logs',
        logging_steps=10,
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        fp16=torch.cuda.is_available(),
        dataloader_num_workers=0
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        eval_dataset=tokenized_dataset
    )

    try:
        trainer.train()
        trainer.save_model('./dialoggpt_model')
        tokenizer.save_pretrained('./dialoggpt_model')
        print('DialogGPT model fine-tuned and saved.')
        return model, tokenizer
    except Exception as e:
        print(f'Error fine-tuning DialogGPT: {e}')
        return None, None

dialoggpt_model, dialoggpt_tokenizer = fine_tune_dialoggpt(dialoggpt_dataset)
if dialoggpt_model is None:
    print('Using pretrained DialogGPT due to fine-tuning error.')
    dialoggpt_model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-medium')
    dialoggpt_tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
    dialoggpt_tokenizer.pad_token = dialoggpt_tokenizer.eos_token


## Step 5: Preprocess for DistilBERT


In [ ]:
def preprocess_data(df):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    
    dataset = Dataset.from_pandas(df[['text', 'intent']])
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset = tokenized_dataset.rename_column('intent', 'labels')
    label_map = {label: idx for idx, label in enumerate(sorted(df['intent'].unique()))}
    tokenized_dataset = tokenized_dataset.map(lambda x: {'labels': label_map[x['labels']]})
    return tokenized_dataset, label_map, tokenizer

dataset, label_map, tokenizer = preprocess_data(df)
print(f'Label Map: {label_map}')


## Step 6: Train DistilBERT Model


In [ ]:
def train_model(dataset, label_map):
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_map))
    train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2).values()
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        fp16=torch.cuda.is_available(),
        dataloader_num_workers=0
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset
    )
    try:
        trainer.train()
        trainer.save_model('./chatbot_model')
        return trainer, model, train_dataset, eval_dataset
    except Exception as e:
        print(f'Error training DistilBERT model: {e}')
        return None, None, None, None

trainer, model, train_dataset, eval_dataset = train_model(dataset, label_map)
if trainer is None:
    raise RuntimeError('Failed to train DistilBERT model.')
with open('label_map.json', 'w') as f:
    json.dump(label_map, f)


## Step 7: Evaluate Model


In [ ]:
def evaluate_model(trainer, eval_dataset, label_map):
    try:
        predictions = trainer.predict(eval_dataset)
        preds = np.argmax(predictions.predictions, axis=1)
        labels = predictions.label_ids
        accuracy = accuracy_score(labels, preds)
        report = classification_report(labels, preds, target_names=label_map.keys())
        print(f'Accuracy: {accuracy:.4f}')
        print(f'Classification Report:\n{report}\n')
        cm = confusion_matrix(labels, preds)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_map.keys(), yticklabels=label_map.keys())
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.show()
    except Exception as e:
        print(f'Error evaluating model: {e}')

evaluate_model(trainer, eval_dataset, label_map)


## Step 8: Dialogue Management


In [ ]:
class DialogueManager:
    def __init__(self, model, tokenizer, label_map, dialoggpt_model, dialoggpt_tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.dialoggpt_model = dialoggpt_model
        self.dialoggpt_tokenizer = dialoggpt_tokenizer
        self.reverse_label_map = {v: k for k, v in label_map.items()}
        self.state = 'INITIAL'
        self.context = defaultdict(str)
        self.history = []
        self.fallback_responses = {
            'greeting': {
                'INITIAL': [
                    'Hello! How can I assist you with your trucking needs today?',
                    'Hi there! What can I help you with regarding your shipments or account?'
                ],
                'POST_GREETING': [
                    'Thanks for the greeting! How can I assist with your shipment or billing needs?',
                    'Nice to connect again! What’s on your mind today?'
                ]
            },
            'farewell': {
                'INITIAL': [
                    'Goodbye! Feel free to reach out if you need further assistance.',
                    'Thanks for connecting! Let me know if you have more questions later.'
                ]
            },
            'small_talk': {
                'INITIAL': {
                    'mood': ['My day’s going smoothly, thanks for asking! How’s yours?'],
                    'plans': ['No big plans here, just helping truckers! Got any big plans yourself?'],
                    'industry': ['Trucking life’s always moving! How’s it treating you these days?'],
                    'default': ['All’s well here, thanks for asking! Need help with your shipments?']
                }
            },
            'compliment': {
                'INITIAL': ['Thank you, that’s kind of you! How can I assist you today?']
            },
            'weather_query': {
                'INITIAL': ['Weather’s clear here, but I can check for your route! Where are you headed?']
            }
        }
        self.trucking_responses = {
            'delivery_status': {
                'INITIAL': 'I can check your shipment status for {location}. Please provide the shipment ID.',
                'AWAITING_SHIPMENT_ID': 'Could you share the shipment ID to proceed with tracking?',
                'PROVIDED_SHIPMENT_ID': 'Thank you. Shipment {shipment_id} is currently at {location}. Would you like the estimated arrival time?'
            },
            'billing_issue': {
                'INITIAL': 'Let’s review your billing issue with {company}. Is this about an overcharge or a payment concern?',
                'AWAITING_DETAILS': 'Can you provide the invoice number or {company} transaction amount?',
                'RESOLVING': 'I’ve noted a {amount} charge on your {company} invoice. Would you like to dispute this?'
            },
            'account_update': {
                'INITIAL': 'I can help update your account details for {regulation}. What information would you like to change?',
                'AWAITING_INFO': 'Please provide the new address or contact details for {regulation}.',
                'CONFIRMING': 'I have {new_info} for your {regulation} update. Please confirm to proceed.'
            },
            'service_inquiry': {
                'INITIAL': 'I can provide information on our services. Are you interested in flatbed, refrigerated, or bulk transport rates?',
                'AWAITING_SPECIFICS': 'Which service are you inquiring about: flatbed, refrigerated, or bulk transport?'
            },
            'fuel_card_query': {
                'INITIAL': 'I can assist with your {company} fuel card or {regulation} query. Is this about a balance, transaction, or compliance?',
                'AWAITING_DETAILS': 'Could you specify if this is a {company} card issue or a {regulation} tax question?'
            },
            'general_query': {
                'INITIAL': 'Could you clarify your request? I can help with delivery, billing, account updates, or {company}/{regulation} services.'
            }
        }

    def predict_intent(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = self.model(**inputs)
            predicted_label = torch.argmax(outputs.logits, dim=1).item()
        return self.reverse_label_map[predicted_label]

    def extract_entities(self, text):
        entities = []
        text = text.lower()
        if 'speedway' in text:
            entities.append({'entity': 'location', 'value': 'Speedway'})
        if 'comdata' in text:
            entities.append({'entity': 'company', 'value': 'Comdata'})
        if 'ifta' in text:
            entities.append({'entity': 'regulation', 'value': 'IFTA'})
        shipment_match = re.search(r'\bship\d+\b', text, re.IGNORECASE)
        if shipment_match:
            entities.append({'entity': 'shipment_id', 'value': shipment_match.group()})
        amount_match = re.search(r'\$\d+', text)
        if amount_match:
            entities.append({'entity': 'amount', 'value': amount_match.group()})
        if 'new address' in text or 'change to' in text:
            new_info = text.split('new address')[-1].strip() or text.split('change to')[-1].strip()
            entities.append({'entity': 'new_info', 'value': new_info[:50]})
        return entities

    def get_dialoggpt_response(self, text):
        try:
            if self.dialoggpt_tokenizer.pad_token is None:
                self.dialoggpt_tokenizer.pad_token = self.dialoggpt_tokenizer.eos_token
            input_ids = self.dialoggpt_tokenizer.encode(text + self.dialoggpt_tokenizer.eos_token, return_tensors='pt')
            reply_ids = self.dialoggpt_model.generate(
                input_ids,
                max_new_tokens=128,
                pad_token_id=self.dialoggpt_tokenizer.eos_token_id,
                no_repeat_ngram_size=3,
                top_p=0.9,
                temperature=0.7,
                do_sample=True
            )
            response = self.dialoggpt_tokenizer.decode(reply_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
            return response
        except Exception as e:
            print(f'Error getting DialogGPT response: {e}')
            return None

    def update_context(self, intent, entities, text):
        self.history.append((intent, text, ''))
        for entity in entities:
            self.context[entity['entity']] = entity['value']
        if intent == 'greeting':
            self.context['greeted'] = 'true'
        if intent == 'small_talk':
            text = text.lower()
            if 'day' in text or 'how’s your day' in text:
                self.context['small_talk_type'] = 'mood'
            elif 'plans' in text or 'big plans' in text:
                self.context['small_talk_type'] = 'plans'
            elif 'trucking' in text or 'truck' in text:
                self.context['small_talk_type'] = 'industry'
            else:
                self.context['small_talk_type'] = 'default'

    def transition_state(self, intent, entities, text):
        if intent in ['greeting', 'small_talk', 'compliment', 'weather_query']:
            self.state = 'POST_GREETING' if self.state == 'INITIAL' and intent == 'greeting' else self.state
        elif intent == 'farewell':
            self.state = 'INITIAL'
        elif intent == 'delivery_status':
            if self.state == 'INITIAL' and 'shipment_id' not in self.context:
                self.state = 'AWAITING_SHIPMENT_ID'
            elif 'shipment_id' in self.context:
                self.state = 'PROVIDED_SHIPMENT_ID'
        elif intent == 'billing_issue':
            if self.state == 'INITIAL' and not any(e['entity'] in ['amount', 'transaction_id'] for e in entities):
                self.state = 'AWAITING_DETAILS'
            elif any(e['entity'] in ['amount', 'transaction_id'] for e in entities):
                self.state = 'RESOLVING'
        elif intent == 'account_update':
            if self.state == 'INITIAL' and 'new_info' not in self.context:
                self.state = 'AWAITING_INFO'
            elif 'new_info' in self.context:
                self.state = 'CONFIRMING'
        elif intent == 'service_inquiry':
            if self.state == 'INITIAL' and 'service_type' not in self.context:
                self.state = 'AWAITING_SPECIFICS'
        elif intent == 'fuel_card_query':
            if self.state == 'INITIAL' and not any(e['entity'] in ['balance', 'transaction'] for e in entities):
                self.state = 'AWAITING_DETAILS'

    def generate_response(self, intent, entities, text):
        self.update_context(intent, entities, text)
        self.transition_state(intent, entities, text)

        if intent in ['greeting', 'farewell', 'small_talk', 'compliment', 'weather_query']:
            dialog_response = self.get_dialoggpt_response(text)
            if not dialog_response or len(dialog_response) < 5 or any(word in dialog_response.lower() for word in ['inappropriate', 'sorry', 'weird', 'lol']):
                small_talk_type = self.context.get('small_talk_type', 'default')
                if intent == 'small_talk':
                    response_options = self.fallback_responses[intent]['INITIAL'].get(small_talk_type, self.fallback_responses[intent]['INITIAL']['default'])
                else:
                    response_options = self.fallback_responses.get(intent, {'INITIAL': ['Could you clarify your request?']}).get(self.state, self.fallback_responses[intent]['INITIAL'])
                response = random.choice(response_options) if isinstance(response_options, list) else response_options
            else:
                response = dialog_response + ' Need assistance with your trucking needs?'
            if intent == 'greeting' and 'good morning' in text.lower() and datetime.now().hour < 12:
                response = random.choice(['Good morning to you too! How can I assist today?', 'Morning! Ready to help with your trucking needs.'])
        else:
            response_template = self.trucking_responses.get(intent, self.trucking_responses['general_query']).get(self.state, self.trucking_responses[intent]['INITIAL'])
            try:
                response = response_template.format(
                    location=self.context.get('location', 'unknown'),
                    company=self.context.get('company', 'unknown'),
                    regulation=self.context.get('regulation', 'unknown'),
                    shipment_id=self.context.get('shipment_id', 'unknown'),
                    amount=self.context.get('amount', 'unknown'),
                    new_info=self.context.get('new_info', 'unknown')
                )
            except KeyError:
                response = response_template

        if self.context.get('greeted') == 'true' and intent not in ['greeting', 'farewell', 'small_talk', 'compliment', 'weather_query'] and 'PROVIDED' in self.state:
            response = f'Since you greeted me earlier, I’m ready to assist! {response}'

        self.history[-1] = (intent, text, response)
        resolved = self.state in ['PROVIDED_SHIPMENT_ID', 'RESOLVING', 'CONFIRMING'] and intent not in ['greeting', 'farewell', 'small_talk', 'compliment', 'weather_query']
        return response, resolved

    def evaluate_dialogue_success(self):
        resolved = sum(1 for intent, _, _ in self.history if intent not in ['greeting', 'farewell', 'small_talk', 'compliment', 'weather_query'] and self.state in ['PROVIDED_SHIPMENT_ID', 'RESOLVING', 'CONFIRMING'])
        total = sum(1 for intent, _, _ in self.history if intent not in ['greeting', 'farewell', 'small_talk', 'compliment', 'weather_query'])
        return resolved / total if total > 0 else 0.0

try:
    with open('label_map.json', 'r') as f:
        label_map = json.load(f)
    dialogue_manager = DialogueManager(model, tokenizer, label_map, dialoggpt_model, dialoggpt_tokenizer)
except FileNotFoundError:
    print('Error: label_map.json not found. Please ensure Cell 6 has run successfully.')
    dialogue_manager = None


## Step 9: Interactive Chatbot UI

UI with DialogGPT conversational support trained on Twitter data.

In [ ]:
if dialogue_manager is None:
    print('Dialogue manager not initialized. Please fix previous errors.')
else:
    # Create UI
    input_box = widgets.Text(
        value='',
        placeholder='Type your query (e.g., Hello, How’s your day?, or Where’s my shipment?)',
        description='Query:',
        layout={'width': '500px'}
    )
    submit_button = widgets.Button(
        description='Submit',
        button_style='success',
        tooltip='Submit query'
    )
    follow_up_button1 = widgets.Button(
        description='Request ETA',
        button_style='info',
        tooltip='Request ETA',
        layout={'visibility': 'hidden'}
    )
    follow_up_button2 = widgets.Button(
        description='Confirm',
        button_style='success',
        tooltip='Confirm action',
        layout={'visibility': 'hidden'}
    )
    output_area = widgets.Output()

    def on_submit_clicked(b):
        with output_area:
            clear_output()
            user_input = input_box.value.strip()
            if not user_input:
                print('Please enter a query.')
                return
            print(f'You: {user_input}')
            try:
                intent = dialogue_manager.predict_intent(user_input)
                entities = dialogue_manager.extract_entities(user_input)
                response, resolved = dialogue_manager.generate_response(intent, entities, user_input)
                print(f'Bot: {response}')
                input_box.value = ''
                follow_up_button1.layout.visibility = 'visible' if dialogue_manager.state == 'PROVIDED_SHIPMENT_ID' else 'hidden'
                follow_up_button2.layout.visibility = 'visible' if dialogue_manager.state in ['RESOLVING', 'CONFIRMING'] else 'hidden'
                success_rate = dialogue_manager.evaluate_dialogue_success()
                if success_rate > 0:
                    print(f'Dialogue Success Rate: {success_rate:.2f}')
            except Exception as e:
                print(f'Error processing query: {e}')

    def on_follow_up1_clicked(b):
        with output_area:
            clear_output()
            response = f'The estimated arrival time for shipment {dialogue_manager.context.get("shipment_id", "unknown")} is tomorrow by 3 PM.'
            dialogue_manager.history.append(('follow_up', 'Request ETA', response))
            print(f'Bot: {response}')
            follow_up_button1.layout.visibility = 'hidden'
            success_rate = dialogue_manager.evaluate_dialogue_success()
            if success_rate > 0:
                print(f'Dialogue Success Rate: {success_rate:.2f}')

    def on_follow_up2_clicked(b):
        with output_area:
            clear_output()
            if dialogue_manager.state == 'CONFIRMING':
                response = f'Confirmed. {dialogue_manager.context.get("new_info", "action")} has been updated.'
            else:
                response = f'Dispute for {dialogue_manager.context.get("amount", "unknown")} has been submitted.'
            dialogue_manager.history.append(('follow_up', 'Confirm', response))
            dialogue_manager.state = 'INITIAL'
            print(f'Bot: {response}')
            follow_up_button2.layout.visibility = 'hidden'
            success_rate = dialogue_manager.evaluate_dialogue_success()
            if success_rate > 0:
                print(f'Dialogue Success Rate: {success_rate:.2f}')

    submit_button.on_click(on_submit_clicked)
    follow_up_button1.on_click(on_follow_up1_clicked)
    follow_up_button2.on_click(on_follow_up2_clicked)

    # Display UI
    display(widgets.VBox([
        widgets.HTML('<h3>Trucking Co. Customer Chatbot</h3>'),
        input_box,
        submit_button,
        follow_up_button1,
        follow_up_button2,
        output_area
    ]))
    print('Welcome! You can greet me, chat about your day, or ask about delivery, billing, or Comdata/IFTA services.')
    print('Type your query and click Submit.')
